In [1]:
# Cargamos las librerias necesarias
import pandas as pd
from pathlib import Path

In [2]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Cargar datos

df_precio_gas = pd.read_csv(f"{BASE_DIR}/Precio_Gas.csv", sep=',', encoding='latin-1')

# (Opcional) Parsear la fecha a datetime y descartar las que no se puedan parsear
df_precio_gas['fecha_de_negociacion'] = pd.to_datetime(df_precio_gas['fecha_de_negociacion'], errors='coerce', dayfirst=True)
df_precio_gas = df_precio_gas[df_precio_gas['fecha_de_negociacion'].notna()]

# Extrae año, mes (1–12) y día (1–31)
df_precio_gas['Year'] = df_precio_gas['fecha_de_negociacion'].dt.year
df_precio_gas['Month'] = df_precio_gas['fecha_de_negociacion'].dt.month
df_precio_gas['Day'] =df_precio_gas['fecha_de_negociacion'].dt.day

# Genera DayofThe Year
df_precio_gas['DayOfYear'] = pd.to_datetime({'year': df_precio_gas['Year'], 'month': df_precio_gas['Month'], 'day': df_precio_gas['Day']}).dt.dayofyear


C:\Users\Elena\AppData\Local\Temp\ipykernel_6372\1442075694.py:8: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_precio_gas['fecha_de_negociacion'] = pd.to_datetime(df_precio_gas['fecha_de_negociacion'], errors='coerce', dayfirst=True)


In [3]:
df_precio_gas_tratado= df_precio_gas[df_precio_gas['producto_de_mibgas'] == 'Producto diario D+1']  
df_precio_gas_tratado= df_precio_gas_tratado[['Year', 'Month', 'Day', 'DayOfYear','precios_de_los_productos_mibgas']].copy()
df_precio_gas_tratado= df_precio_gas_tratado[(df_precio_gas_tratado['Year'] > 2018) & (df_precio_gas_tratado['Year'] < 2024)]

### Añadimos el valor faltante que es 2019, 7, 28 (día 209 ), le imputamos la media 11,4 y 13,07

In [4]:
# Añadir un nuevo registro (al final)
nuevo_registro = {
    'Year': 2019,
    'Month': 7,
    'Day': 28,
    'DayOfYear': 209,
    'precios_de_los_productos_mibgas': (13.07+11.4)/2
}
df_precio_gas_tratado.loc[len(df_precio_gas_tratado)] = nuevo_registro  # se añade al final

# Ordenar por Year, Month, Day
df_precio_gas_tratado = df_precio_gas_tratado.sort_values(
    by=['Year', 'Month', 'Day'],
    ascending=[True, True, True]
).reset_index(drop=True)

df_precio_gas_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             1826 non-null   int64  
 1   Month                            1826 non-null   int64  
 2   Day                              1826 non-null   int64  
 3   DayOfYear                        1826 non-null   int64  
 4   precios_de_los_productos_mibgas  1826 non-null   float64
dtypes: float64(1), int64(4)
memory usage: 71.5 KB


In [5]:
# Guardar resultados
df_precio_gas_tratado.to_csv(f"{BASE_DIR}/P_GAS_TRATADO.csv", index=False)